In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from os.path import abspath

# SparkSession
URL_SPARK = "spark://spark-master:7077"
warehouse_location = abspath('spark-warehouse')

spark = (
    SparkSession.builder
    .appName("write-stream-minio")
    .config("executor.memory", "8g")
    .config("spark.sql.warehouse.dir", warehouse_location)
    .config("spark.jars", "jars/hadoop-aws-3.3.2.jar,jars/aws-java-sdk-bundle-1.12.172.jar,jars/hadoop-common-3.3.2.jar,jars/spark-avro_2.12-3.3.2.jar,jars/commons-pool2-2.11.1.jar")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    .master(URL_SPARK)
    .getOrCreate()
)


In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import unix_timestamp, from_unixtime

In [ ]:
import os
os.environ["AWS_ACCESS_KEY_ID"] = "minioadmin"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minioadmin"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = f"http://sparkjupyter-minio-1:9000"

spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://sparkjupyter-minio-1:9000")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.access.key", "minioadmin")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "minioadmin" )
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled","false")

In [ ]:
path_to_MinIO1 = "s3a://external-acumos-vm-data/"

In [ ]:
df_stream = spark.readStream.text(path_to_MinIO1)

In [ ]:
stringDF = df_stream.selectExpr("CAST(value AS STRING)")

In [ ]:
df= stringDF.withColumn('timestamp', regexp_extract('value', r'timestamp:\s(.*),\sused_memory', 1)) \
        .withColumn('cpu', regexp_extract('value', r'used_cpu:\s(.*)\%', 1)) \
        .withColumn('memory', regexp_extract('value', r'used_memory:\s(.*)\%\s+,\sused_storage', 1)) \
        .withColumn('storage', regexp_extract('value', r'used_storage:\s(.*)\%\s+,\sused_cpu', 1))

In [ ]:
df.writeStream \
    .format("csv") \
    .outputMode("append") \
    .option("path", "s3a://external-acumos-vm-data-transformed/") \
    .option("checkpointLocation", "s3a://external-acumos-vm-data-checkpoint/") \
    .start() \
    .awaitTermination()